# Segmenting and Clustering Neighborhoods in Toronto Canada

### Introduction

In this task, I will work on converting addresses into their equivalent latitude and longitude values. I will also use the Foursquare API to explore neighborhoods in Toronto. Explore the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. 
I'll be using the k-means clustering algorithm to complete this task. 
Finally, I'll use the Folium library to visualize the neighborhoods in Toronto and their emerging clusters.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. [Importing Libraries and Tools](#section1)
    
2. [Download and Explore Dataset](#section2)

3. <a href="#section3">Explore Boroughs in Toronto</a>

4. <a href="#section4">Analyze Each Borough</a>

5. <a href="#section5">Cluster Bouroughs</a>

6. <a href="#section6">Examine Clusters</a>    
</font>
</div>

<a id='section1'></a>
## 1.  Importing Libraries and Tools

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='section2'></a>
## 2. Download and Explore Dataset

In [2]:
url =  'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto = pd.read_html(url)

In [3]:
toronto = toronto[0]
toronto.head()

,Postal Code,District,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
#toronto.rename(columns={'District':'Borough'}, inplace=True)

The first time I did this project, 'District' was called 'Borough'. I do not know the reason for the change. In other to prevent this very little but significant change in naming, affect the remaining parts of the project, I renamed the District column to Borough and saved this dataframeas a CSV file. This csv data will be used for the rest of this project.

In [5]:
toronto = pd.read_csv('toronto_data.csv')
toronto.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
toronto.shape

(180, 3)

In [7]:
toronto.nunique()

Postal Code      180
Borough           11
Neighbourhood    100
dtype: int64

In [8]:
#comment this line of code out when running, the dataframe has already been saved and that's what will be used as we move on

#toronto.to_csv(r"C:\Users\Babatolatemi\Documents\me\coursera\projects\toronto_data.csv", index=False)

**Looking at the number of unique values in each column, it shows that the postal code doesn't contain any duplicate values. Over the years, the Toronto data has been cleaned up, and neighbourhoods with the same postal code are now grouped together**

### Cleaning the data.

1. Rows where borough is 'Not assigned' were dropped
2. If Neighbourhood is 'Not asssigned', the neighbourhood name is the same as the borough name

In [9]:
toronto.drop(toronto[toronto.Borough=='Not assigned'].index, axis=0, inplace=True)

In [10]:
toronto.shape

(103, 3)

In [11]:
for index, row in toronto.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']

In [12]:
toronto.nunique()

Postal Code      103
Borough           10
Neighbourhood     99
dtype: int64

### Getting the Longitude and Latitude

Coursera has provided the coordinate data for toronto. That's what was used here.

In [13]:
coord = pd.read_csv('http://cocl.us/Geospatial_data')
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
coord.shape

(103, 3)

In [15]:
toronto = pd.merge(toronto, coord, how='inner', on='Postal Code')

In [16]:
toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [17]:
toronto.shape

(103, 5)

In [18]:
toronto.isnull().sum()

Postal Code      0
Borough          0
Neighbourhood    0
Latitude         0
Longitude        0
dtype: int64

In [19]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto['Borough'].unique()),
        toronto.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


**Use geopy library to get the latitude and longitude values of Toronto.**

In order to define an instance of the geocoder, a user agent needs to be define.I'll use *toronto_explorer*, as shown below.

In [20]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto boroughs with neighborhoods superimposed on top.

In [21]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood,  borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**I selected only the boroughs with 'Toronto' in their names, there are 39 of them, and plotted them on a map using folium**

In [22]:
borough_with_toronto = []
for bor in toronto['Borough']:
    if 'Toronto' in bor:
        borough_with_toronto.append(bor)

In [23]:
len(borough_with_toronto)

39

In [24]:
borough_with_toronto_df = toronto[pd.DataFrame(toronto.Borough.tolist()).isin(borough_with_toronto).any(1)]
borough_with_toronto_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [25]:
borough_with_toronto_df.shape

(39, 5)

In [26]:
# create map of boroughs with 'Toronto' in their names using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(borough_with_toronto_df['Latitude'], borough_with_toronto_df['Longitude'], 
                           borough_with_toronto_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [27]:
CLIENT_ID = 'NOBZYQP2PWG3LPG0DDHDGS1STD1OW1PL5LG01X2DZPCJM414' # your Foursquare ID
CLIENT_SECRET = 'PKIYHSTKE4QBUVWMZAFFYDTGIMBAQXODZJTYA3NRL5KGM0C0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NOBZYQP2PWG3LPG0DDHDGS1STD1OW1PL5LG01X2DZPCJM414
CLIENT_SECRET:PKIYHSTKE4QBUVWMZAFFYDTGIMBAQXODZJTYA3NRL5KGM0C0


In [28]:
toronto.loc[0, 'Borough']

'North York'

In [29]:
toronto['Neighbourhood'].value_counts()

Downsview                                                                                                                                 4
Don Mills                                                                                                                                 2
North Park, Maple Leaf Park, Upwood Park                                                                                                  1
University of Toronto, Harbord                                                                                                            1
Guildwood, Morningside, West Hill                                                                                                         1
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport                                1
Bayview Village                                                                                                                           1
Moore Park, Summerhi

<a id='section3'></a>

## 3. Explore Boroughs in Toronto

In this part of the notebook, this is where I used the Foursquare API. The API is used to extract venues in the different boroughs using a specific format of search term.

**Steps**

1. From the available dataframe, select the borough of choice, its coordinates and save them to variables.
2. Construct the GET request URL with the Client's API information, the borough information and search term, the search term for this project is **venues**
3. Use the URL created above to make a request to the API (Note: this is a free developer account, there's a limited number of call that can be made daily)
4. The result gotten from the API  is usually in a json format, extract the venue information from this json file and save in a new dataframe.

Practiced using the API with just the first borough to see how it works.

In [30]:
borough_latitude = toronto.loc[0, 'Latitude'] # neighborhood latitude value
borough_longitude = toronto.loc[0, 'Longitude'] # neighborhood longitude value

borough_name = toronto.loc[0, 'Borough'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(borough_name, 
                                                               borough_latitude, 
                                                               borough_longitude))

Latitude and longitude values of North York are 43.7532586, -79.3296565.


#### Getting the top 100 venues that are in North York within a radius of 500 meters.

- First, create the GET request URL.

In [31]:
#setting search constraints
radius = 500   #distance that should be covered by API during search
LIMIT = 100    #maximun num of venues that should be returned per borough

url = """https://api.foursquare.com/v2/venues/explore?&client_id=
{}&client_secret={}&v={}&ll={},{}&radius={}&limit={}""".format(CLIENT_ID, CLIENT_SECRET, VERSION, borough_latitude, 
                                                               borough_longitude, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=\nNOBZYQP2PWG3LPG0DDHDGS1STD1OW1PL5LG01X2DZPCJM414&client_secret=PKIYHSTKE4QBUVWMZAFFYDTGIMBAQXODZJTYA3NRL5KGM0C0&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [32]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f465649f26cab155cfa9906'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

Function to extract the venue category from the json file gotten from the API

In [33]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure it into a pandas dataframe

In [34]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\Babatolatemi\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [35]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


Function to do all the steps above on all the Toronto boroughs.

In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    """INPUT:
            names: borough names
            latitudes, longtitudes: borough coordinates
            radius: search distance
            LIMIT: maximum number of venues to be returned
            
       OUTPUT:
            Dataframe containing all the venues in the boroughs that meet the search criteria
            """
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
toronto_venues = getNearbyVenues(names=toronto['Borough'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )
toronto_venues.head()

North York
North York
Downtown Toronto
North York
Downtown Toronto
Etobicoke
Scarborough
North York
East York
Downtown Toronto
North York
Etobicoke
Scarborough
North York
East York
Downtown Toronto
York
Etobicoke
Scarborough
East Toronto
Downtown Toronto
York
Scarborough
East York
Downtown Toronto
Downtown Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
North York
North York
Scarborough
North York
North York
East Toronto
North York
York
North York
Scarborough
North York
North York
Central Toronto
Central Toronto
York
York
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Etobicoke
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Mississauga
Etobicoke
Scarborough
Central Toronto
Downtown Toronto
West Toron

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North York,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,North York,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,North York,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,North York,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,North York,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [38]:
toronto_venues.shape

(2136, 7)

**Check how many venues were returned for each borough**

In [39]:
toronto_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,111,111,111,111,111,111
Downtown Toronto,1244,1244,1244,1244,1244,1244
East Toronto,126,126,126,126,126,126
East York,70,70,70,70,70,70
Etobicoke,72,72,72,72,72,72
Mississauga,13,13,13,13,13,13
North York,241,241,241,241,241,241
Scarborough,88,88,88,88,88,88
West Toronto,154,154,154,154,154,154


**Find out how many unique categories can be curated from all the returned venues**

In [40]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 275 uniques categories.


<a id='section4'></a>
## 4. Analyze Each Borough

**Used onehot encoding to find all the venue categories in each borough**

In [41]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = toronto_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,North York,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [42]:
toronto_onehot.shape

(2136, 276)

#### Group rows by borough and by taking the mean of the frequency of occurrence of each category

In [43]:
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
toronto_grouped

,Borough,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009009,0.00

#### Print each borough along with the top 5 most common venues

In [44]:
num_top_venues = 5

for b in toronto_grouped['Borough']:
    print("----"+b+"----")
    temp = toronto_grouped[toronto_grouped['Borough'] == b].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.06
2            Café  0.05
3     Pizza Place  0.05
4            Park  0.05


----Downtown Toronto----
                 venue  freq
0          Coffee Shop  0.11
1                 Café  0.06
2           Restaurant  0.03
3                Hotel  0.03
4  Japanese Restaurant  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.06
1         Coffee Shop  0.06
2  Italian Restaurant  0.04
3             Brewery  0.04
4                Café  0.04


----East York----
                 venue  freq
0                 Bank  0.06
1          Coffee Shop  0.06
2         Burger Joint  0.04
3                 Park  0.04
4  Sporting Goods Shop  0.04


----Etobicoke----
            venue  freq
0     Pizza Place  0.11
1  Sandwich Place  0.07
2        Pharmacy  0.06
3   Grocery Store  0.04
4             Gym  0.04


----Mississauga----
                      venue  freq
0                     Hote

#### Put into a *pandas* dataframe

- sort the values in descending order using a function

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

- Create the new dataframe and display the top 10 venues for each neighborhood.

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
borough_venues_sorted = pd.DataFrame(columns=columns)
borough_venues_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
    borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    
borough_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Sushi Restaurant,Restaurant,Dessert Shop,Pub,Health & Beauty Service
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Bar
2,East Toronto,Coffee Shop,Greek Restaurant,Brewery,Italian Restaurant,Café,Ice Cream Shop,Restaurant,Park,Yoga Studio,American Restaurant
3,East York,Coffee Shop,Bank,Sporting Goods Shop,Burger Joint,Sandwich Place,Pizza Place,Park,Grocery Store,Supermarket,Athletics & Sports
4,Etobicoke,Pizza Place,Sandwich Place,Pharmacy,Fast Food Restaurant,Coffee Shop,Gym,Grocery Store,Liquor Store,Fried Chicken Joint,Park


<a id='section5'></a>

## 5. Cluster Boroughs

In [47]:
toronto_grouped

,Borough,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009009,0.00

In [48]:
borough_venues_sorted['Borough'].value_counts()

East Toronto        1
Mississauga         1
Scarborough         1
East York           1
York                1
Downtown Toronto    1
West Toronto        1
Etobicoke           1
Central Toronto     1
North York          1
Name: Borough, dtype: int64

Run *k*-means to cluster the neighborhood into 6 clusters.

In [49]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([3, 3, 3, 0, 5, 1, 3, 4, 3, 2])

Create a new dataframe that includes the cluster as well as the top 10 venues for each borough.

In [50]:
borough_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Sushi Restaurant,Restaurant,Dessert Shop,Pub,Health & Beauty Service
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Bar
2,East Toronto,Coffee Shop,Greek Restaurant,Brewery,Italian Restaurant,Café,Ice Cream Shop,Restaurant,Park,Yoga Studio,American Restaurant
3,East York,Coffee Shop,Bank,Sporting Goods Shop,Burger Joint,Sandwich Place,Pizza Place,Park,Grocery Store,Supermarket,Athletics & Sports
4,Etobicoke,Pizza Place,Sandwich Place,Pharmacy,Fast Food Restaurant,Coffee Shop,Gym,Grocery Store,Liquor Store,Fried Chicken Joint,Park


In [51]:
# add clustering labels
borough_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(borough_venues_sorted.set_index('Borough'), on='Borough')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3,Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Sandwich Place,Pizza Place,Bank,Park,Fast Food Restaurant,Convenience Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,3,Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Sandwich Place,Pizza Place,Bank,Park,Fast Food Restaurant,Convenience Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Bar
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3,Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Sandwich Place,Pizza Place,Bank,Park,Fast Food Restaurant,Convenience Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Bar


Finally, visualize the resulting clusters

In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], 
                                  toronto_merged['Borough'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='section6'></a>
## 6. Examine Clusters

#### Cluster 1

Only borough in this cluster is the East York Borough, but with 5 different postal codes. The most popular venues are all almost identical

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]#, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,0,Coffee Shop,Bank,Sporting Goods Shop,Burger Joint,Sandwich Place,Pizza Place,Park,Grocery Store,Supermarket,Athletics & Sports
14,M4C,East York,Woodbine Heights,43.695344,-79.318389,0,Coffee Shop,Bank,Sporting Goods Shop,Burger Joint,Sandwich Place,Pizza Place,Park,Grocery Store,Supermarket,Athletics & Sports
23,M4G,East York,Leaside,43.709060,-79.363452,0,Coffee Shop,Bank,Sporting Goods Shop,Burger Joint,Sandwich Place,Pizza Place,Park,Grocery Store,Supermarket,Athletics & Sports
29,M4H,East York,Thorncliffe Park,43.705369,-79.349372,0,Coffee Shop,Bank,Sporting Goods Shop,Burger Joint,Sandwich Place,Pizza Place,Park,Grocery Store,Supermarket,Athletics & Sports
35,M4J,East York,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106,0,Coffee Shop,Bank,Sporting Goods Shop,Burger Joint,Sandwich Place,Pizza Place,Park,Grocery Store,Supermarket,Athletics & Sports


#### Cluster 2

The only borough in this cluster is the Mississauga and the 1st most common venue is Hotels.

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
76,M7R,Mississauga,Canada Post Gateway Processing Centre,43.636966,-79.615819,1,Hotel,Coffee Shop,Gym,American Restaurant,Intersection,Middle Eastern Restaurant,Gas Station,Burrito Place,Sandwich Place,Mediterranean Restaurant


#### Cluster 3

The most common venue in this cluster is Convenience Store followed by Park

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,2,Convenience Store,Park,Pizza Place,Discount Store,Bus Line,Pool,Sandwich Place,Field,Hockey Arena,Fast Food Restaurant
21,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,2,Convenience Store,Park,Pizza Place,Discount Store,Bus Line,Pool,Sandwich Place,Field,Hockey Arena,Fast Food Restaurant
56,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013,2,Convenience Store,Park,Pizza Place,Discount Store,Bus Line,Pool,Sandwich Place,Field,Hockey Arena,Fast Food Restaurant
63,M6N,York,"Runnymede, The Junction North",43.673185,-79.487262,2,Convenience Store,Park,Pizza Place,Discount Store,Bus Line,Pool,Sandwich Place,Field,Hockey Arena,Fast Food Restaurant
64,M9N,York,Weston,43.706876,-79.518188,2,Convenience Store,Park,Pizza Place,Discount Store,Bus Line,Pool,Sandwich Place,Field,Hockey Arena,Fast Food Restaurant


#### Cluster 4

North York and Downtown Toronto have been put in the same cluster by KMeans, the most common venues are Coffee Shops

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3,Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Sandwich Place,Pizza Place,Bank,Park,Fast Food Restaurant,Convenience Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,3,Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Sandwich Place,Pizza Place,Bank,Park,Fast Food Restaurant,Convenience Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Bar
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3,Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Sandwich Place,Pizza Place,Bank,Park,Fast Food Restaurant,Convenience Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Bar
7,M3B,North York,Don Mills,43.745906,-79.352188,3,Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Sandwich Place,Pizza Place,Bank,Park,Fast Food Restaurant,Convenience Store
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Bar
10,M6B,North York,Glencairn,43.709577,-79.445073,3,Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Sandwich Place,Pizza Place,Bank,Park,Fast Food Restaurant,Convenience Store
13,M3C,North York,Don Mills,43.725900,-79.340923,3,Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Sandwich Place,Pizza Place,Bank,Park,Fast Food Restaurant,Convenience Store
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Bar


#### Cluster 5

The scarborough borough is the only borough in this cluster and the most common venue here are Fast Food Restaurants

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4,Fast Food Restaurant,Bakery,Bank,Breakfast Spot,Coffee Shop,Intersection,Pizza Place,Chinese Restaurant,Gas Station,Thai Restaurant
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,4,Fast Food Restaurant,Bakery,Bank,Breakfast Spot,Coffee Shop,Intersection,Pizza Place,Chinese Restaurant,Gas Station,Thai Restaurant
18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4,Fast Food Restaurant,Bakery,Bank,Breakfast Spot,Coffee Shop,Intersection,Pizza Place,Chinese Restaurant,Gas Station,Thai Restaurant
22,M1G,Scarborough,Woburn,43.770992,-79.216917,4,Fast Food Restaurant,Bakery,Bank,Breakfast Spot,Coffee Shop,Intersection,Pizza Place,Chinese Restaurant,Gas Station,Thai Restaurant
26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4,Fast Food Restaurant,Bakery,Bank,Breakfast Spot,Coffee Shop,Intersection,Pizza Place,Chinese Restaurant,Gas Station,Thai Restaurant
32,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,4,Fast Food Restaurant,Bakery,Bank,Breakfast Spot,Coffee Shop,Intersection,Pizza Place,Chinese Restaurant,Gas Station,Thai Restaurant
38,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,4,Fast Food Restaurant,Bakery,Bank,Breakfast Spot,Coffee Shop,Intersection,Pizza Place,Chinese Restaurant,Gas Station,Thai Restaurant
44,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,4,Fast Food Restaurant,Bakery,Bank,Breakfast Spot,Coffee Shop,Intersection,Pizza Place,Chinese Restaurant,Gas Station,Thai Restaurant
51,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,4,Fast Food Restaurant,Bakery,Bank,Breakfast Spot,Coffee Shop,Intersection,Pizza Place,Chinese Restaurant,Gas Station,Thai Restaurant
58,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,4,Fast Food Restaurant,Bakery,Bank,Breakfast Spot,Coffee Shop,Intersection,Pizza Place,Chinese Restaurant,Gas Station,Thai Restaurant


#### Cluster 6

Etobicoke is in the 6th cluster and the most popular venues here are Pizza Places

In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,5,Pizza Place,Sandwich Place,Pharmacy,Fast Food Restaurant,Coffee Shop,Gym,Grocery Store,Liquor Store,Fried Chicken Joint,Park
11,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724,5,Pizza Place,Sandwich Place,Pharmacy,Fast Food Restaurant,Coffee Shop,Gym,Grocery Store,Liquor Store,Fried Chicken Joint,Park
17,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,5,Pizza Place,Sandwich Place,Pharmacy,Fast Food Restaurant,Coffee Shop,Gym,Grocery Store,Liquor Store,Fried Chicken Joint,Park
70,M9P,Etobicoke,Westmount,43.696319,-79.532242,5,Pizza Place,Sandwich Place,Pharmacy,Fast Food Restaurant,Coffee Shop,Gym,Grocery Store,Liquor Store,Fried Chicken Joint,Park
77,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,5,Pizza Place,Sandwich Place,Pharmacy,Fast Food Restaurant,Coffee Shop,Gym,Grocery Store,Liquor Store,Fried Chicken Joint,Park
88,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,5,Pizza Place,Sandwich Place,Pharmacy,Fast Food Restaurant,Coffee Shop,Gym,Grocery Store,Liquor Store,Fried Chicken Joint,Park
89,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,5,Pizza Place,Sandwich Place,Pharmacy,Fast Food Restaurant,Coffee Shop,Gym,Grocery Store,Liquor Store,Fried Chicken Joint,Park
93,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,5,Pizza Place,Sandwich Place,Pharmacy,Fast Food Restaurant,Coffee Shop,Gym,Grocery Store,Liquor Store,Fried Chicken Joint,Park
94,M9W,Etobicoke,"Northwest, West Humber - Clairville",43.706748,-79.594054,5,Pizza Place,Sandwich Place,Pharmacy,Fast Food Restaurant,Coffee Shop,Gym,Grocery Store,Liquor Store,Fried Chicken Joint,Park
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,5,Pizza Place,Sandwich Place,Pharmacy,Fast Food Restaurant,Coffee Shop,Gym,Grocery Store,Liquor Store,Fried Chicken Joint,Park
